In [1]:
import pandas as pd
from core.helpers import yakeHelperFunctions as yhf

In [2]:
data_job = pd.read_csv("dataset/job.csv")

In [45]:
vectors = pd.read_csv("word2vec.model", delim_whitespace=True, skiprows=[0], header=None)


In [25]:
# set words as index rather than first column
vectors.index = vectors[0]
vectors.drop(0,axis=1, inplace=True)

In [12]:
data_job.shape

(8165, 1)

# make the sample of 15% of original data

In [13]:
samples_data = data_job.job.sample(frac=.15, random_state=45).astype(str)

# extract keywords & tokenize

In [19]:
keywords = yhf.keywords_yake(samples_data)
keywords_tokenized = yhf.tokenizing_after_yake(keywords)
keywords_set = [ set(job) for job in keywords_tokenized]

# get word vectors  for keywords in jobs description

In [38]:
vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,de,-0.189453,0.181641,0.335938,-0.024048,-0.069824,0.075195,0.128906,-0.341797,-0.074707,...,0.000546,-0.017822,-0.060059,0.343750,-0.184570,0.112305,0.148438,0.116699,0.056885,0.105469
1,et,1.476465,0.650921,1.125034,-0.641027,2.133230,1.009610,0.522009,0.281066,-1.012411,...,0.992428,0.372222,1.381234,-0.745683,0.519218,0.505964,-0.132756,0.549315,1.261921,0.157704
2,des,0.074707,0.112793,0.251953,-0.172852,-0.011658,0.233398,0.349609,-0.219727,-0.060059,...,-0.023071,0.152344,0.111816,0.064453,-0.237305,0.000614,0.107422,0.141602,-0.036377,0.337891
3,la,2.521357,0.398546,0.327319,-0.384584,0.468277,0.268836,0.918718,1.271349,-2.448235,...,1.211048,-0.172852,1.216892,1.828425,0.227053,0.490466,0.163657,0.464512,0.553507,1.263063
4,les,-0.120117,0.095703,0.168945,-0.030273,0.119141,0.147461,0.086426,-0.168945,0.051758,...,0.182617,0.168945,-0.104492,0.033203,-0.488281,-0.053223,0.134766,-0.027832,0.087891,0.289062


In [46]:
def vectors_from_job(job):
        all_words = []
        for words in job:
                all_words.append(words)
        return vectors[vectors.index.isin(all_words)]

In [47]:
vectors_from_job(keywords_set[0])

,1,2,3,4,5,6,7,8,9,10,...,291,292,293,294,295,296,297,298,299,300
0,,,,,,,,,,,,,,,,,,,,,
et,1.476465,0.650921,1.125034,-0.641027,2.133230,1.009610,0.522009,0.281066,-1.012411,1.352205,...,0.992428,0.372222,1.381234,-0.745683,0.519218,0.505964,-0.132756,0.549315,1.261921,0.157704
des,0.074707,0.112793,0.251953,-0.172852,-0.011658,0.233398,0.349609,-0.219727,-0.060059,0.287109,...,-0.023071,0.152344,0.111816,0.064453,-0.237305,0.000614,0.107422,0.141602,-0.036377,0.337891
les,-0.120117,0.095703,0.168945,-0.030273,0.119141,0.147461,0.086426,-0.168945,0.051758,0.308594,...,0.182617,0.168945,-0.104492,0.033203,-0.488281,-0.053223,0.134766,-0.027832,0.087891,0.289062
d,-0.032683,1.842234,0.497303,-0.645244,-0.114350,0.432769,-0.246033,-0.535007,-0.332057,0.421660,...,-0.023890,-1.042520,1.771648,1.573965,0.311247,0.266134,0.238941,0.471029,-0.073571,0.472527
l,1.583955,1.619117,-0.622385,-2.363761,-0.403488,1.468630,0.487609,1.396602,-2.901223,0.504888,...,1.869529,-0.729517,0.116362,0.618401,0.237397,0.493041,0.798146,2.727479,-0.904677,0.298634
dans,2.504843,-1.802795,-0.160548,-0.577735,1.292311,2.761038,-0.422537,-0.827355,-3.269811,1.940815,...,1.006649,-4.029388,0.410320,0.952804,-1.283937,-0.139559,-0.418922,1.380666,-1.645334,1.309486
au,-0.302734,0.114258,0.017700,0.020996,0.047363,0.048340,0.265625,-0.486328,0.051025,0.253906,...,-0.090820,-0.050537,-0.285156,0.200195,-0.103027,-0.015137,-0.065430,-0.094727,-0.039795,0.269531
est,-2.431960,1.350940,0.225080,0.611520,0.783469,1.686311,-2.899148,-0.832241,-2.283552,-3.435134,...,1.160608,4.151691,-0.075979,1.172278,1.406431,-3.854072,-1.665711,3.895567,1.416407,2.547353
projet,-0.168945,-0.071777,0.131836,-0.071289,-0.118652,0.094727,0.152344,-0.164062,-0.055176,0.142578,...,-0.040039,0.023804,-0.088867,0.082520,0.009277,-0.048096,-0.110840,-0.018433,-0.079102,0.072754


In [48]:
# test out getting job description embedding (in the dumbest way possible)

test_vectors = vectors_from_job(keywords_set[0])
test_vectors.mean()

1      0.201541
2      0.110994
3      0.086030
4     -0.241969
5      0.340148
         ...   
296    0.051756
297    0.305604
298    0.529300
299    0.030205
300    0.194320
Length: 300, dtype: float64